In [1]:
from dspy.datasets import MATH

dataset = MATH(subset='algebra')
print(len(dataset.train), len(dataset.dev))

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1187/1187 [00:00<00:00, 208923.16 examples/s]

350 350


<bound method MATH.metric of <dspy.datasets.math.MATH object at 0x7f05d6dc6f80>>

In [21]:

import dspy
import os
from dotenv import load_dotenv
load_dotenv()
base_url = 'https://integrate.api.nvidia.com/v1'
model = "meta/llama-3.3-70b-instruct"

llama33_70b = dspy.LM(f'openai/{model}', api_key=os.environ["NVIDIA_API_KEY"], api_base=base_url)

mixtral8x22b = dspy.LM('openai/nvdev/mistralai/mixtral-8x22b-instruct-v0.1', api_key=os.environ["NV_DEV_API_KEY"], api_base=base_url, temperature=0.7)

dspy.configure(lm=mixtral8x22b)

In [ ]:
## read in nvbug example excel sheet

In [27]:
import pandas as pd
#xls = pd.ExcelFile('/workspace/nvdata/Eval data set.xlsx')
#df1 = pd.read_excel(xls, 'Single Bug Details')
train=pd.read_csv('/workspace/nvdata/train.csv')
train.head()



,Unnamed: 0,startphrase,ending0,ending1,ending2,ending3,label
0,0,Which factor will most likely cause a person to develop a fever?,a leg muscle relaxing after exercise,a bacterial population in the bloodstream,several viral particles on the skin,carbohydrates being digested in the stomach,1
1,1,Lichens are symbiotic organisms made of green algae and fungi. Wha...,carbon dioxide,food,protection,water,1
2,2,"When a switch is used in an electrical circuit, the switch can",cause the charge to build.,increase and decrease the voltage.,cause the current to change direction.,stop and start the flow of current.,3
3,3,Which of the following is an example of an assistive device?,contact lens,motorcycle,raincoat,coffee pot,0
4,4,"Rocks are classified as igneous, metamorphic, or sedimentary accor...",their color,their shape,how they formed,the minerals they contain,2


In [8]:
## filter to get only the accurate examples
correct_col=list(df1.columns)[-1]
df=df1[df1[correct_col]=="Accurate"]
df

,Query,API Response,Response Time (sec),Unnamed: 3
0,what is the reason for delay in this bug fix f...,"{'Category': 'detailed', 'count': 0, 'list': [...",7.143266,Accurate
4,"Can you suggest if a bug 4504063 can be closed,","{'Category': 'detailed', 'count': 0, 'list': [...",3.785950,Accurate
7,list all delays in responses between comments ...,"{'Category': 'detailed', 'count': 0, 'list': [...",5.738200,Accurate
8,Who was the last person to comment on Bug 4323...,"{'Category': 'detailed', 'count': 0, 'list': [...",2.580094,Accurate
9,What changes were made in Bug 4323093 in the l...,"{'Category': 'detailed', 'count': 0, 'list': [...",3.815290,Accurate
10,"Can Bug 4323093 be marked as resolved?,","{'Category': 'detailed', 'count': 0, 'list': [...",3.152200,Accurate
11,"Who is the current QA Engineer for Bug 4323093?,","{'Category': 'detailed', 'count': 0, 'list': [...",2.482323,Accurate
12,"What is the current status of Bug 4323093?,","{'Category': 'detailed', 'count': 0, 'list': [...",3.656847,Accurate
13,"Which module is Bug 4323093 associated with?,","{'Category': 'detailed', 'count': 0, 'list': [...",2.447568,Accurate
15,"What is the severity level of Bug 4323093?,","{'Category': 'detailed', 'count': 0, 'list': [...",2.429880,Accurate


In [28]:
import dspy
from dspy.datasets import HotPotQA

## can replace with anything that return a list of texts, e.g retrieved from RAG
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=500).train]
react = dspy.ReAct("question -> answer", tools=[search_wikipedia])


Generating test split: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7405/7405 [00:01<00:00, 4764.60 examples/s]


In [29]:
trainset[0]

Example({'question': 'Are Smyrnium and Nymania both types of plant?', 'answer': 'yes'}) (input_keys={'question'})

In [ ]:
search_wikipedia("David Gregory was born in 1625.")

In [11]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(declare=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("declare")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:20], hover[20:40], hover[40:60]

[]

In [12]:
example = trainset[0]

example

Example({'declare': 'This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".', 'titles': ['Miss Potter', 'Academy Award for Best Director', 'Chris Noonan']}) (input_keys={'declare'})

In [13]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [23]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the declaration."
signature = dspy.Signature("declare -> titles: list[str]", instructions)
print("signagure >>> ", signature )
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

signagure >>>  StringSignature(declare -> titles
    instructions='Find all Wikipedia titles relevant to verifying (or refuting) the declaration.'
    declare = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Declare:', 'desc': '${declare}'})
    titles = Field(annotation=list[str] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Titles:', 'desc': '${titles}'})
)


In [24]:
react(declare="David Gregory was born in 1625.").titles[:3]

['David Gregory (physician)']

In [15]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    print('**pred titles **',[x for x in pred.titles[:5] ])
    print("'\n'")
    print('---> gold titles', [x for x in gold_titles])
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5, verbose=True)

In [19]:
def safe_react(declare: str):
    try:
        return react(declare=declare)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

  0%|                                                                                                                                                                                     | 0/20 [00:00<?, ?it/s]**pred titles ** []
'
'
---> gold titles ['Ternstroemia', 'Cussonia', 'Seemannaralia gerrardii']
Average Metric: 0.00 / 1 (0.0%):   5%|███████                                                                                                                                     | 1/20 [00:30<09:42, 30.63s/it]**pred titles ** []
'
'
---> gold titles ['Chen Xiuke', 'Danyang, Jiangsu', 'Dongfang, Hainan']
Average Metric: 0.00 / 2 (0.0%):  10%|██████████████                                                                                                                              | 2/20 [00:32<04:09, 13.85s/it]**pred titles ** ["Daniel Boone's Trail", 'Arcadia, Tennessee', 'Green-Jones War', 'National Road']
'
'
---> gold titles ['Arcadia, Tennessee', 'Greene–Jones War', 'Wilderness Road

2025/01/08 14:54:26 INFO dspy.evaluate.evaluate: Average Metric: 6.0 / 20 (30.0%)


,declare,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,Jonathan Howsmon Davis lives closer to Canada than this Japanese s...,"[Jonathan Davis, Singing Bird, Koshi Inaba]","{'thought_0': ""The first task is to identify the Japanese singer w...",The task was to find relevant Wikipedia titles to verify the decla...,[Yui (singer)],
1,An actor born in 1955 that acted in the movie that was the inspira...,"[Ransom (1996 film), Gary Sinise, Blackmail (2005 film)]",{'thought_0': 'The declaration consists of multiple parts: the bir...,The declaration is composed of three main parts: an actor born in ...,"[Actors born in 1955, Movies that inspired Blackmail (2005 film), ...",
2,The star of Nothing to Report and Gary Barlow have a profession in...,"[Nothing to Report, Chris Jericho, Gary Barlow]","{'thought_0': 'To verify this declaration, I need to find the prof...","To verify the declaration, I needed to find the professions of bot...","[Dawn French, Gary Barlow]",✔️ [0.333]
3,"Neither the director of the 1943 film ""Young and Willing"" nor Edwa...","[Edward H. Griffith, Edward Burns, Young and Willing]",{'thought_0': 'I need to verify the nationality of the director of...,I attempted to find the Wikipedia page for the director of the fil...,[],
4,"The actor, who starred in the thriller 400 Boys, plays Marilyn Mon...","[400 Boys, Charlie Rowe, The Golden Compass (film)]","{'thought_0': 'To verify this declaration, I need to find the acto...","To verify the given declaration, I would need to find the actor wh...","['400 Boys', 'Actor who played in 400 Boys', 'Marilyn Monroe film ...",✔️ [0.333]


30.0

In [20]:
dspy.inspect_history()





[2025-01-08T14:54:26.968628]

System message:

Your input fields are:
1. `declare` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## declare ## ]]
{declare}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## titles ## ]]
{titles}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Find all Wikipedia titles relevant to verifying (or refuting) the declaration.


User message:

[[ ## declare ## ]]
The director of action movie Batman: Mask of the Phantasm, produced Avengers Assemble that premiered on Disney XD on May 26, 2013.

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
The declaration suggests that the director of "Batman: Mask of the

In [15]:
kwargs = dict(teacher_settings=dict(lm=llama33_70b), prompt_model=mixtral8x22b, max_errors=999)

tp = dspy.MIPROv2(metric=top5_recall, auto="light", num_threads=16,verbose=True, **kwargs)
optimized_react = tp.compile(react, trainset=trainset, max_bootstrapped_demos=3, max_labeled_demos=0)

2025/01/07 18:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 3
valset size: 16



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 3 * 2 lm calls in program + (3) lm calls in program-aware proposer = 19 prompt model calls
- Program Evaluation: 16 examples in val set * 7 batches = 112 LM program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:

- Reducing the numb

2025/01/07 18:45:41 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/07 18:45:41 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/07 18:45:41 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=3 sets of demonstrations...


Bootstrapping set 1/3
Bootstrapping set 2/3



 25%|███████████████████████████████████████▎                                                                                                                     | 1/4 [00:35<01:46, 35.62s/it]

...


Predictor 0
i: Find all Wikipedia titles relevant to verifying (or refuting) the claim.

You will be given `claim` and your goal is to finish with `titles`.

To do this, you will interleave Thought, Tool Name, and Tool Args, and receive a resulting Observation.

Thought can reason about the current situation, and Tool Name can be the following types:

(1) search_wikipedia, whose description is <desc>Returns top-5 results and then the titles of the top-5 to top-30 results.</desc>. It takes arguments {'query': 'str'} in JSON format.
(2) lookup_wikipedia, whose description is <desc>Returns the text of the Wikipedia page, if it exists.</desc>. It takes arguments {'title': 'str'} in JSON format.
(3) finish, whose description is <desc>Signals that the final outputs, i.e. `titles`, are now available and marks the task as complete.</desc>. It takes arguments {} in JSON format.
p: Next Tool Args:
Predictor 1
i: Utilize all available Wikipedia resources to thoroughly investigate and gather evid

2025/01/07 18:56:42 INFO dspy.evaluate.evaluate: Average Metric: 4.0 / 16 (25.0%)
2025/01/07 18:56:42 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 25.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1', 'Predictor 1: Instruction 1', 'Predictor 1: Few-Shot Set 1'].
2025/01/07 18:56:42 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [27.08, 18.75, 25.0]
2025/01/07 18:56:42 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 27.08
2025/01/07 18:56:42 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/07 18:56:42 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 3 / 7 =====
2025/01/07 18:56:42 INFO dspy.teleprompt.mipro_optimizer_v2: Evaluating the following candidate program...




Predictor 0
i: Given a claim, your task is to identify relevant Wikipedia titles that can be used to verify or refute the claim. You will do this by generating a sequence of thoughts, tool names, and tool arguments, and then executing the corresponding tools to retrieve information from Wikipedia. Your final output should be a list of Wikipedia titles that are relevant to the claim.

To begin, analyze the claim and generate a thought that describes the initial situation and your plan for identifying relevant titles. Then, choose a tool and provide the necessary arguments for executing that tool. Repeat this process until you have identified all relevant titles, and then use the `finish` tool to signal that the task is complete.

Remember to follow the instructions carefully and use the available tools appropriately. Your output should include the fields `next_thought`, `next_tool_name`, and `next_tool_args`.
p: Next Tool Args:
Predictor 1
i: You are a fact-checking assistant for a hig

2025/01/07 18:57:50 WARNING dspy.utils.parallelizer: Received SIGINT. Cancelling execution.
[W 2025-01-07 18:57:56,560] Trial 2 failed with parameters: {'0_predictor_instruction': 2, '0_predictor_demos': 2, '1_predictor_instruction': 2, '1_predictor_demos': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/workspace/dspy/utils/parallelizer.py", line 179, in _execute_multi_thread
    for future in as_completed(futures):
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 245, in as_completed
    waiter.event.wait(wait_timeout)
  File "/usr/lib/python3.10/threading.py", line 607, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 320, in wait
    waiter.acquire()
  File "/workspace/dspy/utils/parallelizer.py", line 136, in interrupt_handler
    default_handler(sig, frame)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent ca

**pred titles ** ['Beethoven', 'Luigi Cherubini', 'Irrelohe']
'
'
---> gold titles ['Luigi Cherubini', 'Franz Schreker', 'Irrelohe']
**pred titles ** ['Michael Noonan']
'
'
---> gold titles ['Leader of Fine Gael', 'Leo Varadkar', 'Michael Noonan']
**pred titles ** ['David Bowman (botanist)', 'Carlina']
'
'
---> gold titles ['David Bowman (botanist)', 'Dieffenbachia', 'Carlina']
**pred titles ** ['Wiliwili tree', 'Acacia koa']
'
'
---> gold titles ['Acacia koa', 'Hawaiian tropical dry forests', 'Wiliwili']
**pred titles ** ['Zak Ové', 'A. Edward Sutherland']
'
'
---> gold titles ['Horace Ové', 'Zak Ové', 'A. Edward Sutherland']
**pred titles ** ['50th Berlin International Film Festival', 'Golden Bear award', 'List of films awarded the Golden Bear']
'
'
---> gold titles ['Magnolia (film)', '50th Berlin International Film Festival', 'Jason Robards']
**pred titles ** ['David Nixon', 'Rhinestone']
'
'
---> gold titles ['Phil Alden Robinson', 'David Nixon (director)', 'Rhinestone (film)']


In [16]:
dspy.inspect_history()






[2025-01-07T18:58:42.876950]

System message:

Your input fields are:
1. `claim` (str)
2. `trajectory` (str)

Your output fields are:
1. `reasoning` (str)
2. `titles` (list[str])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## titles ## ]]
{titles}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        You are a fact-checking assistant for a high-profile legal case. Your task is to find all Wikipedia titles that are relevant to verifying or refuting the claim. This is a critical step in building a strong case, so it's important to be thorough and accurate. Use your language model capabilities to generate a sequence of thoughts, tool names, and tool arguments th